This file is largely intended to discover the circular (non-neural) molecular fingerprinting and drawing capacities of the RDKit library. Unfortunately, the online documentation is largely lacking.

In [1]:
import sys
print(sys.executable)

/home/sohompaul/anaconda3/envs/rdkit-env/bin/python


In [2]:
import matplotlib
from matplotlib import pyplot as plt

import numpy as np

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray

In [3]:
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

Using TensorFlow backend.


In [4]:
from spektral.datasets import qm9
from spektral.utils import label_to_one_hot
from spektral.chem import numpy_to_rdkit

In [5]:
A, X, E, y = qm9.load_data(
    return_type='numpy',
    nf_keys='atomic_num',
    ef_keys='type',
    self_loops=True,
    amount=None
)

Loading QM9 dataset.
Reading SDF


In [6]:
count = 0
for a, x, e in zip(A, X, E):
    if count < 5:
        mol = numpy_to_rdkit(a, x, e)
        # Draw.MolToMPL(mol)
        print(Chem.MolToSmiles(mol))
    count += 1

C
N
O
C#C
C#N


In [7]:
mol = Chem.MolFromSmiles('C(=O)(N)N')

In [8]:
# Draw.MolToMPL(mol)

In [9]:
fingerprint = GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)

In [10]:
fp_array = np.zeros((1, ))
ConvertToNumpyArray(fingerprint, fp_array)

In [11]:
print(fp_array)
print(fp_array.shape)

[0. 0. 0. ... 0. 0. 0.]
(2048,)


## We now experiment with learning on molecular fingerprints.

In [14]:
A_sample, X_sample, E_sample = list(), list(), list()
fingerprint_sample = list()
y_sample = y.sample(10000)
for ind, row in y.iterrows():
    A_sample.append(A[ind])
    X_sample.append(X[ind])
    E_sample.append(E[ind])

In [15]:
for a, x, e in zip(A_sample, X_sample, E_sample):
    mol = numpy_to_rdkit(a, x, e)
    print(Chem.MolToSmiles(mol))
    fingerprint = GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    fp_array = np.zeros((1, ))
    ConvertToNumpyArray(fingerprint, fp_array)
    fingerprint_sample.append(fp_array)
fingerprint_sample = np.stack(fingerprint_sample)

C


RuntimeError: Pre-condition Violation
	getNumImplicitHs() called without preceding call to calcImplicitValence()
	Violation occurred on line 153 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_implicitValence > -1
	RDKIT: 2017.09.1
	BOOST: 1_63
